# Скрипт отвечает за загрузку, предобработку и добавление новых данных по номенклатуре в справочник базы данных PostgreSQL

# 1. IMPORTS

In [1]:
import os
import sys
from functools import lru_cache

import pandas as pd
from sqlalchemy import Engine
from loguru import logger

sys.path.insert(0, r'C:\Users\user\Desktop\github\sotrans_analytics_department')
from source.core.de.connectors import PGConnector
from source.utils.decorators.handlers.excel import handler_xl_shared_string_xml
from source.utils.dtypes import set_category_dtype

# 2. SETTINGS

In [2]:
logger.remove()
logger.add(
    sink=sys.stderr,
    level='TRACE',
)

1

# 3. CONSTANTS

In [3]:
ROOT_FOLDER: str = r'C:\Users\user\YandexDisk\batch_movement\reference'
FILENAME: str = 'nomenclature.xlsx'

In [4]:
ENGINE: Engine = PGConnector().engine

2025-05-19 11:42:27.522 | INFO     | source.core.de.connectors.postgresql:__create_engine:94 - Начало создания подключения к базе данных.
2025-05-19 11:42:27.523 | INFO     | source.core.de.connectors.postgresql:__create_connection_string:61 - Начало формирования строки подключения к базе данных.
2025-05-19 11:42:27.525 | SUCCESS  | source.core.de.connectors.postgresql:__create_connection_string:73 - Строка подключения к базе данных успешно создана.
2025-05-19 11:42:27.562 | SUCCESS  | source.core.de.connectors.postgresql:__create_engine:97 - Подключение к базе данных успешно создано.


# 4. FUNCTIONS

In [5]:
@lru_cache(maxsize=128)
@handler_xl_shared_string_xml
def create_dataframe() -> pd.DataFrame:
    """
    Функция создает DataFrame с данными из файла, используя библиотеку Pandas.

    Returns:
        pd.DataFrame: DataFrame с данными из файла.
    """
    
    dataframe: pd.DataFrame = pd.read_excel( # type: ignore
        io=os.path.join(ROOT_FOLDER, FILENAME),
        engine='openpyxl',
        dtype='string'
    )
    return dataframe

# 5. ETL

## 5.1. Extract

In [6]:
# Создать DataFrame с данными из файла;
dataframe: pd.DataFrame = create_dataframe()

In [7]:
# Замена пропусков на строки '_нет данных';
dataframe.fillna('_нет данных', inplace=True) # type: ignore

## 5.2. Transform

In [8]:
for column in dataframe.columns:
    try:
        logger.info(f'Преобразование строк столбца "{column}" к нижнему регистру;')
        dataframe[column] = dataframe[column].str.lower() # type: ignore
        logger.success(f'Преобразование строк столбца "{column}" к нижнему регистру выполнен;')
    
    except Exception as e:
        logger.error(f'Непредвиденная ошибка: {e};')

2025-05-19 11:42:58.829 | INFO     | __main__:<module>:3 - Преобразование строк столбца "brand_id_1c" к нижнему регистру;
2025-05-19 11:42:58.967 | SUCCESS  | __main__:<module>:5 - Преобразование строк столбца "brand_id_1c" к нижнему регистру выполнен;
2025-05-19 11:42:58.968 | INFO     | __main__:<module>:3 - Преобразование строк столбца "brand_name_1c" к нижнему регистру;
2025-05-19 11:42:59.043 | SUCCESS  | __main__:<module>:5 - Преобразование строк столбца "brand_name_1c" к нижнему регистру выполнен;
2025-05-19 11:42:59.044 | INFO     | __main__:<module>:3 - Преобразование строк столбца "sku_id_1c" к нижнему регистру;
2025-05-19 11:42:59.218 | SUCCESS  | __main__:<module>:5 - Преобразование строк столбца "sku_id_1c" к нижнему регистру выполнен;
2025-05-19 11:42:59.219 | INFO     | __main__:<module>:3 - Преобразование строк столбца "sku_name_1c" к нижнему регистру;
2025-05-19 11:42:59.395 | SUCCESS  | __main__:<module>:5 - Преобразование строк столбца "sku_name_1c" к нижнему регистр

In [9]:
# Установка категориальных типов данных;
dataframe: pd.DataFrame = set_category_dtype(
    dataframe=dataframe,
    columns=tuple(dataframe.columns)
)

2025-05-19 11:42:59.755 | TRACE    | source.utils.dtypes.set_dtypes:set_category_dtype:33 - Количество строк в DataFrame: 450192;
2025-05-19 11:42:59.756 | TRACE    | source.utils.dtypes.set_dtypes:set_category_dtype:34 - Размер трети DataFrame: 150064
2025-05-19 11:42:59.757 | TRACE    | source.utils.dtypes.set_dtypes:set_category_dtype:45 - Тип данных для столбца "brand_id_1c": string.
2025-05-19 11:42:59.853 | TRACE    | source.utils.dtypes.set_dtypes:set_category_dtype:54 - Количество уникальных значений: 3026.
2025-05-19 11:42:59.854 | DEBUG    | source.utils.dtypes.set_dtypes:set_category_dtype:58 - Количество уникальных значений менее трети строк датафрейма;
2025-05-19 11:42:59.854 | INFO     | source.utils.dtypes.set_dtypes:set_category_dtype:59 - Преобразование столбца "brand_id_1c" к категориальному типу данных;
2025-05-19 11:42:59.999 | SUCCESS  | source.utils.dtypes.set_dtypes:set_category_dtype:64 - Столбец "brand_id_1c" преобразован к категориальному типу данных;
2025-05-

In [11]:
# Получить текущие данные из базы данных PostgreSQL;
with ENGINE.begin() as connection:
    pg_nomenclature: pd.DataFrame = pd.read_sql_table(
        table_name='nomenclature',
        con=connection,
        schema='constant',
    )

In [12]:
# Удалить из текущего DataFrame те строки, которые уже присутствуют в базе данных PostgreSQL;
filtered_dataframe: pd.DataFrame = (
    dataframe[
        ~dataframe['sku_id_1c'].isin(values=pg_nomenclature['sku_id_1c'].unique()) # type: ignore
    ]
)

## 5.3. Load

In [13]:
with ENGINE.begin() as connection:
    # Вставить отфильтрованный DataFrame в базу данных PostgreSQL;
    filtered_dataframe.to_sql(
        name='nomenclature',
        schema='constant',
        con=connection,
        index=False,
        if_exists='append',
    )